# BINANCE BOT PYTHON
*****
Author - Автор кода: Alex Klimov   
Link - Ссылка на код: https://www.youtube.com/watch?v=Lqd5szEQ52Y&list=PLdoiuMB5MAYznx6oLfUNoFs__IdtUk0BO&index=2   
Date(s): 26/07/2024 - 30/07/2024   
Development Period: 25/07/2024 - 30/07/2024   
Summary Developing Time - Затраченное время:   

## SPECIFICATION / ОПИСАНИЕ
*****
Программа, с помощью импортированных(!) библиотек python-binance, pandas:   
   
1.1 - подключается к бирже Binance, используя API keys и Client (из библиотеки python-binance);   
   
def top_coin():   
2.1 - собирает и фильтрует спотовые тикеры (в паре с USDT), исключая тикеры, включающие 'UP' и 'DOWN';   
2.2 - определяет/ищет самую активную пару за день (top_coin);   
   
def last_data():   
3.1 - собирает исторические данные по свечам;   
3.2 - проверят находится-ли пара в росте;   
*3.3 - выводит название тикера (top_coin);   
   
def strategy():   
4.0 - Функция по умолчанию имеет значения:   
- buy_amt (количество USDT, для закупа)   
- SL (стоп-лосс)   
- Target (цель/профит)   
- open_position (значение, определяющее статус торгов: FALSE - не торгуется, TRUE - торговля открыта)   
   
4.1 - проверяем получил-ли бот данные;   
4.1.1 - если нет, то делаем паузу на 61 секунду и перезапускаем;   
(?) 4.2 - определяем/указываем значение закупаемого объема (qty):   
*****   
``` python   
qty = round(buy_amt/df.Close.iloc[-1] ,1)   
```   

[Объем (закупаемое_количество_коинов) = округленное_значение(n$ разделенное на последнюю_стоимость_коина)]   
[ где n$ - количество USDT, которые мы готовы продать/обменять/использовать в сделке = баланс (?)]   
   
*Например:*   
Для пары ACE/USDT, которая на момент покупки торговалась по 3.2, при закупе на 10$   
объем будет составлять: round( 10 / 3.2 ) = 3.125 (???)   
*****   
4.3 - проверяем, если цена закрытия последней свечи больше, чем предпоследняя (актив растет);   
4.4 - распечатываем данные (тикер, цена последней закрытой свечи, объем закупаемых монет);   
4.5 - окрываем ордер на покупку; (*НАДО ДОБАВИТЬ ПРОВЕРКУ ДЛЯ БАЛАНСА) !!!   
4.5.1 - определяем переменную buyprice = цена покупки/цена входа;   
4.5.2 - открываем позицию/запускаем торговлю (open_position = True);   
4.6 - каждую минуту (61 секунда) мониторим цену и распечатываем:   
- цену в моменте входа;   
- целевую цену, которую мы ждем;   
- напоминание, при котором должен сработать СТОП-ЛОСС (!)   

4.7 - определяем/указываем/задаем условие, при котором позиция должна закрыться:   
*****   
- df.Close[-1] = buyprice * SL   
или   
- df.Close[-1] = buyprice * Target   
где df.Close[-1] - значение последней закрытой свечи (?)   
*****   
4.8 - если подходящей пары не найдено, то объявляется перекур на 20 (секунд/минут ???)   
4.9 - запуск бота с указанным балансом для торгов   

# Код с комментариями:
_может содержать ошибки и опечатки_   
***перед запуском необходимо установить необходимые библиотеки***   

In [ ]:
pip install python-binance, pandas, time

In [ ]:
from binance.client import Client
import keys
import pandas as pd
import time

# =============== API ===============
# Подключение API и Secret ключей, выданных биржей Binance
client = Client(keys.api_key, keys.api_secret)


# =============== ОПРЕДЕЛЕНИЕ САМОЙ АКТИВНОЙ МОНЕТЫ ===============
# Создаем функцию для поиска самой активной монеты
def top_coin():
  # Получаем все тикеры
  all_tickers = pd.DataFrame(client.get_ticker())
  # Определяем переменную usdt - из всех тикеров выбираются только те, которые содержат значение 'USDT'
  usdt = all_tickers[all_tickers.symbol.str.contains('USDT')]
  # Отсеиваем лишние тикеры, где монеты содержат значение 'UP' и 'DOWN', из-за "низкого торгового процента"
  work = usdt[~((usdt.symbol.str.contains('UP')) | (usdt.symbol.str.contains('DOWN'))) ]
  # Создаем список и сортируем монеты по процентному росту от максимальной к минимальной 
  # и определяем самую активную из них, присваивая значение top_coin
  top_coin = work[work.priceChangePercent == work.priceChangePercent.max()]
  top_coin = top_coin.symbol.values[0]
  return top_coin

# print(top_coin()) # Должен выводить тот самый тикер   


# =============== ПРОВЕРКА КОИНА НА РОСТ ===============
# Создаем функцию для проверки данных по 3 параметрам:
# symbol - исследуемый актив для торгов
# interval - интервал данных/торгов для анализа
# lookback - на сколько минут назад анализировать данные
def last_data(symbol, interval, lookback):

  frame = pd.DataFrame(client.get_historical_klines(symbol, interval, lookback + 'min ago UTC'))
  frame = frame.iloc[:,:6]
  frame.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
  frame = frame.set_index('Time')
  frame.index = pd.to_datetime(frame.index, unit='ms')
  frame = frame.astype(float)
  return frame


# =============== ТОРГОВАЯ СТРАТЕГИЯ ===============
# Определяем функцию с параметрами:
# buy_amt - объем на который мы будем заходить в сделку, 
# SL (StopLoss), значение можно менять по своему усмторению, в данном случае равен -1.5%
# Target (Target profit), значение можно менять по своему усмторению, в данном случае равен х1.02 (2%)
# open_position - аттрибут обозначающий состояние торговли False (по умолчанию) - торговля остановлена, True - запуск торговли
def strategy(buy_amt, SL=0.985, Target=1.02, open_position=False):
  # Запускаем функцию в "проверочном режиме"
  try:
    asset = top_coin()
    df = last_data(asset, '1m', '120')

  # если запуск при проверке не состоялся по каким-либо причинам, то:
  # timesleep(time) - перерыв(значение для интервала), в данном коде 61 = 1 минута
  # затем "повторить попытку"
  except:
    time.sleep(61) # предлагаем боту поспать минуту
    asset = top_coin()
    df = last_data(asset, '1m', '120')

  # Определяем значение объема с которым заходим в позицию = qty
  # qty равно округленному значению из вычислений:
  # buy_amt = Объем разделенный на последняя "цена на момент закрытия"
  # Условно при закупе на 10$ >>>
  # мы закупемся на 10$/ последняя стоимость коина
  qty = round(buy_amt/df.Close.iloc[-1] ,1)

  # Проверяем, если цена закрытия последней свечи больше, чем предпоследняя (актив растет)
  if ((df.Close.pct_change() + 1).cumprod()).iloc[-1] > 1:
    # распечатываем актив (тикер) - чтобы видеть
    print(asset)
    # распечатываем цену закрытия послелней свечи
    print(df.Close.iloc[-1])
    # распечатываем объем = количество закупаемых монет
    print(qty)
    # =============== ОРДЕР ===============
    # Создаем ордер на покупку (Открываем ордер)
    # asset = top_coin
    # Side - направление ПОКУПКА или ПРОДАЖА = BUY/SELL
    # type = стратегия, в данном видео стратегия MARKET ???
    # quantity = qty - закупаемый объем
    order = client.create_order(symbol=asset, side='BUY', type='MARKET', quantity = qty)
    # Распечатываем ордер
    print(order)
    # Определяем переменную buyprice = цена покупки/цена входа
    buyprice = float(order['fills'][0]['price'])
    # Запускаем торговлю (Входим в позицию/торги)
    open_position = True

    # Контроль позиции/цены коина
    while open_position:
      try:
        df = last_data(asset, '1m', '2')
      except:
        print('Минуточку...')
        time.sleep(61) # предлагаем боту поспать минуту
        df = last_data(asset, '1m', '2')

      # Для контроля/мониторинга 
      # Распечатываем цену в моменте входа
      print(f'ЦЕНА ' + str(df.Close[-1]))
      # Распечатываем целевую цену, которую мы ждем
      print(f'ЦЕЛЬ ' + str(buyprice * Target))
      # Распечатываем напоминание при котором будет СТОП
      print(f'СТОП ' + str(buyprice * SL))

      # =============== ЗАКРЫТИЕ ОРДЕРА ===============
      # Проверяем/Задаем условия для закрытия сделки
      if df.Close[-1] <= buyprice * SL or df.Close[-1] >= buyprice * Target:
        # Закрываем ордер
        order = client.create_order(symbol=asset, side='SELL', type='MARKET', quantity = qty)
        # Распечатываем ордер
        print(order)
        break # Останавливаем цикл

  # ЕСЛИ нет подходящих монет
  # Перекур!!!
  else:
    print('Покури пока, ничего не найдено')
    time.sleep(20)

# strategy(15) - объем на который мы закупаемся = 15$
while True:
  strategy(15)

## ОРИГИНАЛЬНЫЙ КОД АВТОРА РОЛИКА

In [ ]:
pip install python-binance
pip install pandas
pip instal time

In [ ]:
from binance.client import Client
import keys
import pandas as pd
import time

client = Client(keys.api_key, keys.api_secret)


def top_coin():

    all_tickers = pd.DataFrame(client.get_ticker())
    usdt = all_tickers[all_tickers.symbol.str.contains('USDT')]
    work = usdt[~((usdt.symbol.str.contains('UP')) | (usdt.symbol.str.contains('DOWN'))) ]
    top_coin = work[work.priceChangePercent == work.priceChangePercent.max()]
    top_coin = top_coin.symbol.values[0]
    return top_coin




def last_data(symbol, interval, lookback):

    frame = pd.DataFrame(client.get_historical_klines(symbol, interval, lookback + 'min ago UTC'))
    frame = frame.iloc[:,:6]
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    frame = frame.set_index('Time')
    frame.index = pd.to_datetime(frame.index, unit='ms')
    frame = frame.astype(float)
    return frame



def strategy(buy_amt, SL=0.985, Target=1.02, open_position=False):
    try:
        asset = top_coin()
        df = last_data(asset, '1m', '120')
    except:
        time.sleep(61)
        asset = top_coin()
        df = last_data(asset, '1m', '120')
        
    qty = round(buy_amt/df.Close.iloc[-1] ,1)
    
    if ((df.Close.pct_change() + 1).cumprod()).iloc[-1] > 1:
        print(asset)
        print(df.Close.iloc[-1])
        print(qty)
        order = client.create_order(symbol=asset, side='BUY', type='MARKET', quantity = qty)
        print(order)
        buyprice = float(order['fills'][0]['price'])
        open_position = True
        
        while open_position:
            try:
                df = last_data(asset, '1m', '2')
            except:
                print('Restart after 1 min')
                time.sleep(61)
                df = last_data(asset, '1m', '2')
            
            print(f'Price ' + str(df.Close[-1]))
            print(f'Target ' + str(buyprice * Target))
            print(f'Stop ' + str(buyprice * SL))
            if df.Close[-1] <= buyprice * SL or df.Close[-1] >= buyprice * Target:
                order = client.create_order(symbol=asset, side='SELL', type='MARKET', quantity = qty)
                print(order)
                break
    else:
        print('No find')
        time.sleep(20)

while True:
    strategy(15)

# ЗАПУСК БОТА
*****
   
***!!! Для запуска в VS Code, необходимо предварительно в терминале/консоле, установить и активировать виртуальную среду:***   
   
``` shell   
python -m venv .venv    # создание виртуальной среды
.venv\Scripts\activate    # активация виртуальной среды
```   
   
**Устанавливаем необходимые библиотеки**   
   
``` shell   
pip install python-binance   
pip install pandas   
pip install time   
```   
   
_если вознукнут проблемы с установкой библиотек (см. bug_report-01-26072024), можно воспользоваться дополнительной командой --user:_   
   
``` shell   
pip install --user python-binance   
pip install --user pandas   
pip install --user time   
```   
   
Запуск бота осуществляется нажатием кнопки ['>' Run Code] или командой в терминале/консоле:   
``` shell   
python file_name.py   
```   


# ТЕСТЫ
*****

### Test-case #1: Предварительная проверка (Первый Запуск)   
   
IDEA: Первый запуск бота (Проверка на работоспособность)   
Date: 26/07/2024   
TC ID: BBP001   
Priority: 1   
AUTHOR: Unsigable   
GLOBAL SETUP and ADDITIONAL INFO:   
- run project file >>> python main.py   
*****   
   
**EXECUTION PART (ИСПОЛНЕНИЕ)**   
   
***PROCEDURE: (ШАГИ)***   
1. open vs code;   
2. create file;   
3. python -m venv .venv   
4. .venv\Scripts\activate   
5. add code;   
6. add API_KEY & SECRET_KEY;   
7. python main.py   
   
***EXPECTED RESULT: (ОЖИДАЕМЫЙ РЕЗУЛЬТАТ)***   
- Запуск бота   
- Вывод в терминале "самой активной пары" (top_coin):   
``` SHELL
OUTPUT:   
Наименование_тикера   
```   
   
***ACTUAL RESULT: (ФАКТИЧЕСКИЙ РЕЗУЛЬТАТ)***   
ERROR (ОШИБКА - см. bug_report-01-26072024):   
   
``` SHELL   
OUTPUT:   
Traceback (most recent call last):
  File "d:\UMAR\DEVELOPMENT\TUTORIALS\2024\PYTHON\Money_Maker\2 - Binance Bot Python (Alex Klimov)\main.py", line 24, in <module>
    from binance.client import Client
ModuleNotFoundError: No module named 'binance
```   

#### ПЕРВЫЙ БАГ
*****
**BUG_REPORT-01-26072024**   
   
При первом запуске бота, программа выдала ошибку:   
   
``` SHELL   
Traceback (most recent call last):
  File "d:\UMAR\DEVELOPMENT\TUTORIALS\2024\PYTHON\Money_Maker\2 - Binance Bot Python (Alex Klimov)\main.py", line 24, in <module>
    from binance.client import Client
ModuleNotFoundError: No module named 'binance
```   
   
**ОПИСАНИЕ ОШИБКИ**   
Суть ошибки: программа не видела бибилотеку binance, несмотря на то, что та была предварительно установлена   
Причина ошибки: в VS Code не выбрана или не верно указана виртуальная среда разработки (.venv)   
   
**СТАТУС ОШИБКИ:**   
РЕШЕНО/УСТРАНЕНА (FIXED)   
СПОСОБ РЕШЕНИЯ: Заново повторил шаги 1-7, НО проследил, чтобы .venv был указан верно (относился к папке проекта!)

### Test-case #2: Предварительная проверка (Второй Запуск)   
   
IDEA: Второй запуск бота (Проверка на работоспособность)   
Date: 27/07/2024   
TC ID: BBP002   
Priority: 1   
AUTHOR: Unsigable   
GLOBAL SETUP and ADDITIONAL INFO:   
- run project file >>> python main.py   
*****   
   
**EXECUTION PART (ИСПОЛНЕНИЕ)**   
   
***PROCEDURE: (ШАГИ)***   
1. open vs code;   
2. create file;   
3. python -m venv .venv   
4. .venv\Scripts\activate   
5. add code;   
6. add API_KEY & SECRET_KEY;   
7. python main.py   
   
***EXPECTED RESULT: (ОЖИДАЕМЫЙ РЕЗУЛЬТАТ)***   
- Запуск бота   
- Вывод в терминале "самой активной пары" (top_coin):   
``` SHELL
OUTPUT:   
Наименование_тикера   
```   
   
***ACTUAL RESULT: (ФАКТИЧЕСКИЙ РЕЗУЛЬТАТ)***   
   
``` SHELL   
OUTPUT:   
VANRYUSDT   
```   

### Test-case #3: Проверка вывода тикетов на повторение (Третий Запуск)   
   
IDEA: Проверить какую пару покажет бот при другом запуске, туже самую или другую   
Date: 27/07/2024   
TC ID: BBP003   
Priority: 2   
AUTHOR: Unsigable   
GLOBAL SETUP and ADDITIONAL INFO:   
- run project file >>> python main.py   
*****   
   
**EXECUTION PART (ИСПОЛНЕНИЕ)**   
   
***PROCEDURE: (ШАГИ)***   
1. python main.py   
   
***EXPECTED RESULT: (ОЖИДАЕМЫЙ РЕЗУЛЬТАТ)***   
- Запуск бота   
- Вывод в терминале "самой активной пары" (top_coin) другой или той-же самой:   
``` SHELL
OUTPUT:   
Наименование_тикера   
```   
   
***ACTUAL RESULT: (ФАКТИЧЕСКИЙ РЕЗУЛЬТАТ)***   
   
``` SHELL   
OUTPUT:   
THETAUSDT   
```   

### Test-case #4: Проверка вывода тикетов по заявленному алгоритму   
   
IDEA: Проверить действительно-ли бот выбирает самую активную за день пару на бирже   
Date: 27/07/2024   
TC ID: BBP004   
Priority: 1   
AUTHOR: Unsigable   
GLOBAL SETUP and ADDITIONAL INFO:   
- run project file >>> python main.py   
- Данные биржи Binance;   
*****   
   
**EXECUTION PART (ИСПОЛНЕНИЕ)**   
   
***PROCEDURE: (ШАГИ)***   
1. .venv\Scripts\activate
2. python main.py   
3. Сравнить с данными биржи Binance;
4. Повторить шаги 1 - 3 еще 9 раз;
   
***EXPECTED RESULT: (ОЖИДАЕМЫЙ РЕЗУЛЬТАТ)***   
- Запуск бота   
- Вывод в терминале "самой активной пары" (top_coin):   
``` SHELL   
OUTPUT:   
Наименование_тикера ###USDT   
```   
- Данная пара действительно является на момент запуска самой активной на бирже за сутки/день торгов (!);
   
***ACTUAL RESULT: (ФАКТИЧЕСКИЙ РЕЗУЛЬТАТ)***   
   
``` TXT   
OUTPUTs:   
№1 - PUNDIXUSDT - 10 место в списке бирже (% за 24 часа по убыванию) - 21:55   
№2 - PUNDIXUSDT - 9 место в списке бирже (% за 24 часа по убыванию) - 21:56   
№3 - PUNDIXUSDT - 9 место в списке бирже (% за 24 часа по убыванию) - 21:57   
№4 - PUNDIXUSDT - 9 место в списке бирже (% за 24 часа по убыванию) - 21:58   
№5 - JASMYUSDT - 9 место в списке бирже (% за 24 часа по убыванию) - 21:59   
№6 - PUNDIXUSDT - 9 место в списке бирже (% за 24 часа по убыванию) - 22:00   
№7 - PUNDIXUSDT - 9 место в списке бирже (% за 24 часа по убыванию) 22:01   
№8 - PUNDIXUSDT - 9 место в списке бирже (% за 24 часа по убыванию) 22:02   
№9 - PUNDIXUSDT - 9 место в списке бирже (% за 24 часа по убыванию) 22:03   
№10 - JASMYUSDT - 10 место в списке бирже (% за 24 часа по убыванию) 22:04   
```   
   
В топ-10 монет по % за 24 часа (на момент тестов):   
1 - STMX ~ 41%   
2 - XEC ~ 14.6%   
3 - STORJ ~ 14%   
4 - RIF ~ 14%   
5 - ACE ~ 12%   
6 - CFX ~ 11%   
7 - AMB ~ 10%   
8 - ???   
9 - PUNDIX ~ 9.85%   
10 - JASMY ~ 9.5%   
   

In [1]:
from binance.client import Client
import keys
import pandas as pd
import time


client = Client(keys.api_key, keys.api_secret)


# ================== ПОИСК САМОЙ АКТИВНОЙ МОНЕТЫ ==================
def top_coin():

    all_tickers = pd.DataFrame(client.get_ticker())
    usdt = all_tickers[all_tickers.symbol.str.contains('USDT')]
    work = usdt[~((usdt.symbol.str.contains('UP')) | (usdt.symbol.str.contains('DOWN'))) ]
    top_coin = work[work.priceChangePercent == work.priceChangePercent.max()]
    top_coin = top_coin.symbol.values[0]
    return top_coin

print(top_coin()) # распечатываем самую активную монету

COMPUSDT


### Test-case #5: Проверка открытия и закрытия ордеров   
   
IDEA: Проверить как бот открывает и закрывает ордера   
Date: 27/07/2024   
TC ID: BBP005   
Priority: 1   
AUTHOR: Unsigable   
GLOBAL SETUP and ADDITIONAL INFO:   
- run project file >>> python main.py   
- Аккаунт на бирже Binance;   
*****   
   
**EXECUTION PART (ИСПОЛНЕНИЕ)**   
   
***PROCEDURE: (ШАГИ)***   
1. .venv\Scripts\activate
2. python main.py   
3. Открытие ордера;
4. Проверить аккаунт на бирже Binance;
4. Дождаться закрытия ордера;
   
***EXPECTED RESULT: (ОЖИДАЕМЫЙ РЕЗУЛЬТАТ)***   
- Запуск бота   
- Поиск активной пары   
- Открытие ордера   
- Закрытие ордера (при достижении цены одного из условий закрытия)   
   
***ACTUAL RESULT: (ФАКТИЧЕСКИЙ РЕЗУЛЬТАТ)***   
- Бот запустился   
- Бот выбрал активную пару   
- Бот открыл ордер   
- При достижении порога (STOP-LOSS) Бот остановился и выдал ошибку   
``` SHELL   
raise BinanceAPIException(response, response.status_code, response.text)   
binance.exceptions.BinanceAPIException: APIError(code=-1013): Filter failure: LOT_SIZE   
```   
   

#### ВТОРОЙ БАГ
*****
**BUG_REPORT-02-27072024**   
   
При втором запуске бота, в момент, когда бот должен был закрыть позицию из-за достижения порога (STOP-LOSS) программа выдала ошибку:   
   
``` SHELL   
raise BinanceAPIException(response, response.status_code, response.text)   
binance.exceptions.BinanceAPIException: APIError(code=-1013): Filter failure: LOT_SIZE   
```   
   
**ОПИСАНИЕ ОШИБКИ**   
Суть ошибки: Бот не смог совершить операцию по закрытию позиции (создание ордера на продажу)    
Причина ошибки: нехватка монет для закрытия позиции   
   
**СТАТУС ОШИБКИ:**   
РЕШЕНО/УСТРАНЕНА (FIXED)   
СПОСОБ РЕШЕНИЯ: Внёс изменнения в код. Теперь при закупе и продаже, бот:   
- Бот докупает на 0.2% монет больше (чтобы покрыть коммиссионные)   
- Бот продает на 0.2% монет меньше (чтобы учесть коммиссионные)   
- ВВОДИТЬ/УКАЗЫВАТь (buy_amt) > 5.0
   

### СОВЕТЫ CHATGPT
*****
   
Ошибка BinanceAPIException: APIError(code=-1013): Filter failure: LOT_SIZE возникает, когда параметры вашего заказа не соответствуют ограничениям, установленным биржей Binance. Эта ошибка чаще всего связана с несоответствием количества заказа (количества монет) минимальному или максимальному допустимому значению или шагу для указанного торгового пара.   
   
Вот шаги, которые помогут вам устранить эту ошибку:   
   
1. Проверка LOT_SIZE для торгового пары: Узнайте, какие ограничения на количество действуют для вашего торгового пара. Это можно сделать, запросив информацию о биржевых правилах через API или проверив их на сайте Binance.   
   
2. Соответствие количества ордера правилам: Убедитесь, что количество монет в вашем ордере соответствует требованиям LOT_SIZE. Например, если минимальное количество для ордера 0.001 и шаг 0.001, ваш ордер должен быть кратен этому шагу (например, 0.001, 0.002, 0.003 и т.д.).   
   
3. Пример кода для проверки LOT_SIZE:   
   
``` python   
from binance.client import Client
from binance.exceptions import BinanceAPIException

# Инициализация клиента
api_key = 'your_api_key'
api_secret = 'your_api_secret'
client = Client(api_key, api_secret)

# Получение информации о правилах биржи
exchange_info = client.get_exchange_info()

# Находим нужный торговый пар
symbol = 'BTCUSDT'
symbol_info = next((item for item in exchange_info['symbols'] if item['symbol'] == symbol), None)

if symbol_info:
    for filter in symbol_info['filters']:
        if filter['filterType'] == 'LOT_SIZE':
            min_qty = float(filter['minQty'])
            max_qty = float(filter['maxQty'])
            step_size = float(filter['stepSize'])
            print(f"Минимальное количество: {min_qty}")
            print(f"Максимальное количество: {max_qty}")
            print(f"Шаг: {step_size}")
else:
    print(f"Торговый пар {symbol} не найден.")
```   
   
4. Корректировка ордера: После того как вы узнаете ограничения, скорректируйте параметры своего ордера, чтобы они соответствовали этим требованиям.   
   
*****

##### ПОЛУЧЕНИЕ ДАННЫХ О ТОРГОВОЙ ПАРЕ (minQTY)

In [2]:
from binance.client import Client
from binance.exceptions import BinanceAPIException
import keys

# Инициализация клиента
client = Client(keys.api_key, keys.api_secret)
# api_key = 'your_api_key'
# api_secret = 'your_api_secret'
# client = Client(api_key, api_secret)

# Получение информации о правилах биржи
exchange_info = client.get_exchange_info()

# Находим нужную торговую пару (Укажите нужную)
symbol = 'COMPUSDT' 
symbol_info = next((item for item in exchange_info['symbols'] if item['symbol'] == symbol), None)

if symbol_info:
    for filter in symbol_info['filters']:
        if filter['filterType'] == 'LOT_SIZE':
            min_qty = float(filter['minQty'])
            max_qty = float(filter['maxQty'])
            step_size = float(filter['stepSize'])
            print(f"Минимальное количество: {min_qty}")
            print(f"Максимальное количество: {max_qty}")
            print(f"Шаг: {step_size}")
else:
    print(f"Торговый пар {symbol} не найден.")


Минимальное количество: 0.001
Максимальное количество: 90000.0
Шаг: 0.001


*****
Ошибка BinanceAPIException: APIError(code=-2010): Account has insufficient balance for requested action указывает на то, что на вашем аккаунте недостаточно средств для выполнения запрашиваемой операции. Это может быть связано с недостаточным количеством средств на вашем счету для покупки или продажи выбранного актива.   
   
Вот шаги, которые помогут вам устранить эту ошибку:   
   
1. Проверка баланса аккаунта: Убедитесь, что у вас достаточно средств для выполнения операции. Вы можете сделать это, проверив баланс через API или на сайте Binance.   
   
2. Пример кода для проверки баланса:   
   
``` python   
from binance.client import Client
from binance.exceptions import BinanceAPIException

# Инициализация клиента
api_key = 'your_api_key'
api_secret = 'your_api_secret'
client = Client(api_key, api_secret)

# Получение баланса аккаунта
try:
    balance = client.get_asset_balance(asset='BTC')  # Замените 'BTC' на нужный вам актив
    print(balance)
except BinanceAPIException as e:
    print(f"Ошибка при получении баланса: {e}")
```   
   
   
3. Проверка объема ордера: Убедитесь, что объем ордера (количество активов) не превышает доступное количество на вашем счету. Например, если у вас на счету 0.5 BTC, а вы пытаетесь продать 1 BTC, то это вызовет ошибку.   
   
4. Корректировка ордера: Если у вас недостаточно средств, уменьшите количество активов в ордере или пополните баланс на своем аккаунте.   
   
5. Пример кода для создания ордера с учетом баланса:   
   
``` python
from binance.client import Client
from binance.exceptions import BinanceAPIException

# Инициализация клиента
api_key = 'your_api_key'
api_secret = 'your_api_secret'
client = Client(api_key, api_secret)

# Проверка и создание ордера
try:
    # Проверка баланса
    balance = client.get_asset_balance(asset='BTC')  # Замените 'BTC' на нужный вам актив
    available_balance = float(balance['free'])
    
    # Определите количество, которое вы хотите продать
    quantity_to_sell = 0.1  # Пример значения
    
    # Проверьте, достаточно ли у вас средств
    if available_balance >= quantity_to_sell:
        # Создайте ордер
        order = client.order_market_sell(
            symbol='BTCUSDT',  # Замените на нужный вам торговый пар
            quantity=quantity_to_sell
        )
        print(order)
    else:
        print("Недостаточно средств для выполнения ордера")
except BinanceAPIException as e:
    print(f"Ошибка при создании ордера: {e}")
```   
   
*****

##### ПОЛУЧЕНИЕ ДАННЫХ О ТОРГОВОЙ ПАРЕ (minQTY)

In [3]:
from binance.client import Client
from binance.exceptions import BinanceAPIException
import keys

# Инициализация клиента
client = Client(keys.api_key, keys.api_secret)

# Получение баланса аккаунта
try:
    balance = client.get_asset_balance(asset='USDT')  # Замените 'USDT' на нужный вам актив
    print(balance)
except BinanceAPIException as e:
    print(f"Ошибка при получении баланса: {e}")

Ошибка при получении баланса: APIError(code=-2015): Invalid API-key, IP, or permissions for action.


## ДОПОЛНЕННЫЙ БОТ

In [ ]:
from binance.client import Client
import keys
import pandas as pd
import time


client = Client(keys.api_key, keys.api_secret)

print('ВАЖНО! Необходимо учитывать: ')
print('* Коммиссия за каждую операцию (покупка/продажа) составляет - 0.1%')
print('* Для некоторых монет минимальный объем (minQTY): 5 коинов')
print('* При вводе суммы помните про STOP-LOSS и КОММИССИЮ х2 (0.2%)')
print('* МИНИМАЛЬНАЯ РЕКОМЕНДУЕМАЯ СУММА: 10 коинов')
buy_amt = float(input("ВВЕДИТЕ СУММУ ПЛАНИРУЕМОЙ СДЕЛКИ: ")) # ВВОД СУММЫ (USDT)



# ======================== ПОИСК АКТИВНОЙ МОНЕТЫ ========================
def top_coin():

    all_tickers = pd.DataFrame(client.get_ticker())
    coin = all_tickers[all_tickers.symbol.str.contains('USDT')]
    work = coin[~((coin.symbol.str.contains('UP')) | (coin.symbol.str.contains('DOWN'))) ]
    top_coin = work[work.priceChangePercent == work.priceChangePercent.max()]
    top_coin = top_coin.symbol.values[0]
    return top_coin

print(top_coin()) # распечатываем выбранную ботом монету



# ======================== ИСТОРИЧЕСКИЕ ДАННЫЕ ========================
def last_data(symbol, interval, lookback):

    frame = pd.DataFrame(client.get_historical_klines(symbol, interval, lookback + 'min ago UTC'))
    frame = frame.iloc[:,:6]
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    frame = frame.set_index('Time')
    frame.index = pd.to_datetime(frame.index, unit='ms')
    frame = frame.astype(float)
    return frame



# ======================== ТОРГОВАЯ СТРАТЕГИЯ ========================
def strategy(buy_amt, SL=0.985, Target=1.02, open_position=False):
    try:
        asset = top_coin()
        df = last_data(asset, '1m', '120')
    except:
        time.sleep(61)
        asset = top_coin()
        df = last_data(asset, '1m', '120')

    # ======================== ОБЪЕМ  ========================
    qty = round(buy_amt/df.Close.iloc[-1] ,1) # закупаемый объем монет
    qty_o = round(qty + (qty * 0.2 / 100)) # закупаемый объем + 0.2% коммиссионные
    qty_c = round(qty - (qty * 0.2 / 100)) # закупаемый объем + 0.2% коммиссионные

    # ================== ДАННЫЕ О МОНЕТЕ ==================
    info = client.get_symbol_info(asset)
    print(f' ================== ДАННЫЕ О МОНЕТЕ ================== ')
    print(info) # Выводим данные о монете
    # Узнаем минимальный объем (USDT) для открытия ордера = minQty (LOT_SIZE)
    minQty = info['filters'][1]['minQty']
    # Узнаем минимальный объем (USDT) для закрытия ордера = minNotional (NOTIONAL)
    # minNotional определяет минимальную условную стоимость, необходимую для каждого заказа
    minNotional = info['filters'][6]['minNotional']
    print(f' ====================================================== ')

    print(f' ')
    print(f'ПАРА: ' + str(asset))
    print(f'ПОСЛЕДНЯЯ ЦЕНА: ' + str(df.Close.iloc[-1]))
    print(f'БЮДЖЕТ НА СДЕЛКУ: ' + str(buy_amt))
    print(f'ЗАКУПАЕМЫЙ ОБЪЕМ МОНЕТ: ' + str(qty))
    print(f'МИНИМАЛЬНЫЙ ОБЪЕМ (USDT) НА ЗАКУП: ' + str(minQty))
    print(f'МИНИМАЛЬНЫЙ ОБЪЕМ (USDT) ДЛЯ ЗАКРЫТИЯ СДЕЛКИ: ' + str(minNotional))
    print(f'ОЖИДАЕМЫЙ БАЛАНС МОНЕТ после вычета КОММИССИИ: ' + str(qty - (qty * 0.1 / 100)))
    print(f' ')

    # ======================== ОТКРЫТИЕ ОРДЕРА ========================
    if ((df.Close.pct_change() + 1).cumprod()).iloc[-1] > 1:
        print(f'КОИН: ' + str(asset))
        print(f'ЦЕНА ПОКУПКИ: ' + str(df.Close.iloc[-1]))
        print(f'ЗАКУПЛЕННЫЙ ОБЪЕМ: ' + str(qty_o))
        order = client.create_order(symbol=asset, side='BUY', type='MARKET', quantity = qty_o)
        print(order)
        buyprice = float(order['fills'][0]['price'])

        trade_qty = float(order['fills'][0]['qty'])
        commission = float(order['fills'][0]['commission'])
        balance = trade_qty - commission
        usdt_in_c = balance * df.Close.iloc[-1]
        print(f' ----------------------- ')
        print(f'ORDER-info: ')
        print(f'ЦЕНА ПОКУПКИ: ' + str(buyprice))
        print(f'ЦЕЛЬ: ' + str(buyprice * Target))
        print(f'СТОП: ' + str(buyprice * SL))
        print(f'БАЛАНС МОНЕТ: ' + str(balance))
        print(f'ОБЪЕМ МОНЕТ: ' + str(trade_qty))
        print(f'ОБЪЕМ МОНЕТ в USDT: ' + str(usdt_in_c))
        print(f'КОММИССИЯ: ' + str(commission))
        print(f' ----------------------- ')
        open_position = True
        # ======================== ЗАКРЫТИЕ ОРДЕРА ========================
        while open_position:
            try:
                df = last_data(asset, '1m', '2')
            except:
                print('ПЕРЕРЫВ на МИНУТОЧКУ!')
                time.sleep(61)
                df = last_data(asset, '1m', '2')
            
            print(f' ')
            print(f'Монета (asset): ' + str(asset))
            print(f'Баланс (trade_qty): ' + str(trade_qty))
            print(f'Баланс в USDT (usdt_in_c): ' + str(usdt_in_c))
            print(f'Цена сейчас: ' + str(df.Close.iloc[-1]))
            print(f'Target ' + str(buyprice * Target))
            print(f'Stop ' + str(buyprice * SL))
            if df.Close[-1] <= buyprice * SL or df.Close[-1] >= buyprice * Target:
                order = client.create_order(symbol=asset, side='SELL', type='MARKET', quantity = qty_c)
                print(order)
                break
    else:
        print('ПОДХОДЯЩЕЙ ПАРЫ НЕ НАЙДЕНО!')
        time.sleep(20)
# ЗАПУСК СТРАТЕГИИ
while True:
    strategy(buy_amt)

#### ТРЕТИЙ БАГ
*****
**BUG_REPORT-03-30072024**   
   
При очередном тестовом запуске бота (чтобы убедиться в том, что он способен закрывать позиции), программа выдала ошибку:   
   
``` SHELL   
Traceback (most recent call last):
  File "D:\UMAR\DEVELOPMENT\TUTORIALS\2024\PYTHON\Money_Maker\2.1 - Binance Bot Python (Alex Klimov)\run.py", line 129, in <module>un.py", line 129, in <module>
    while True:
        ^^^^^^^^                                                                                      un.py", line 121, in strategy
  File "D:\UMAR\DEVELOPMENT\TUTORIALS\2024\PYTHON\Money_Maker\2.1 - Binance Bot Python (Alex Klimov)\r df.Close.iloc[]un.py", line 121, in strategy
    if df.Close.iloc[-1] <= buyprice * SL or df.Close.iloc[-1] >= buyprice * Target: # РЕКОМЕНДУЕТСЯ -venv\Lib\site-packages\binance\client.py", line 1448, in cr df.Close.iloc[]
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\UMAR\DEVELOPMENT\TUTORIALS\2024\PYTHON\Money_Maker\2.1 - Binance Bot Python (Alex Klimov)\.venv\Lib\site-packages\binance\client.py", line 1448, in create_order                                 venv\Lib\site-packages\binance\client.py", line 418, in _po
    return self._post('order', True, data=params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\UMAR\DEVELOPMENT\TUTORIALS\2024\PYTHON\Money_Maker\2.1 - Binance Bot Python (Alex Klimov)\.venv\Lib\site-packages\binance\client.py", line 418, in _post                                         venv\Lib\site-packages\binance\client.py", line 378, in _re
    return self._request_api('post', path, signed, version, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\UMAR\DEVELOPMENT\TUTORIALS\2024\PYTHON\Money_Maker\2.1 - Binance Bot Python (Alex Klimov)\.venv\Lib\site-packages\binance\client.py", line 378, in _request_api                                  venv\Lib\site-packages\binance\client.py", line 359, in _re
    return self._request(method, uri, signed, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\UMAR\DEVELOPMENT\TUTORIALS\2024\PYTHON\Money_Maker\2.1 - Binance Bot Python (Alex Klimov)\.venv\Lib\site-packages\binance\client.py", line 359, in _request                                      venv\Lib\site-packages\binance\client.py", line 368, in _ha
    return self._handle_response(self.response)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "D:\UMAR\DEVELOPMENT\TUTORIALS\2024\PYTHON\Money_Maker\2.1 - Binance Bot Python (Alex Klimov)\.uested action.venv\Lib\site-packages\binance\client.py", line 368, in _handle_response                              )>
    raise BinanceAPIException(response, response.status_code, response.text)   
binance.exceptions.BinanceAPIException: APIError(code=-2010): Account has insufficient balance for requested action.   
```   
   
**ОПИСАНИЕ ОШИБКИ**   
Ошибка BinanceAPIException: APIError(code=-2010): Account has insufficient balance for requested action указывает на то,   
что на вашем аккаунте недостаточно средств для выполнения запрашиваемой операции.   
Это может быть связано с недостаточным количеством средств на вашем счету для покупки или продажи выбранного актива.   
   
**СТАТУС ОШИБКИ:**   
???   
   